In [1]:
import json
from openai import OpenAI

from homeworks.utils.aidevsApi import get_auth_token, get_task, send_answer, API_KEY_OPENAI
from homeworks.utils.additional import download_json_data_from_url


client = OpenAI(api_key=API_KEY_OPENAI)

token = get_auth_token('people')
task = get_task(token, print_task=True)
question = task['question']

----------- Task description -----------
msg: retrieve the data set (JSON) and answer the question. The question will change every time the task is called. I only ask about favourite colour, favourite food and place of residence
data: https://zadania.aidevs.pl/data/people.json
question: jaki kolor się podoba Mariuszowi Kaczorowi?
hint1: Does everything have to be handled by the language model?
hint2: prepare knowledge DB for this task
-----------    ----------    -----------


In [2]:
url = "https://zadania.aidevs.pl/data/people.json"
people_data = download_json_data_from_url(url)

JSON data downloaded successfully.


In [3]:
# ask LLM about name and surname
response = client.chat.completions.create(
    model="gpt-4-0613",
    messages=[
        {"role": "system", "content": f"Poniżej będzie pytanie od użytkownika. Wyciągnij z niego informacje o imieniu"
                                      f"i nazwisku osoby w pytaniu. Jeśli imię nie ma formy podstawowej imienia to je zmień."
                                      f"Na przykład 'Mirka' zmien na 'Mirosław', 'Tomek' na 'Tomasz', 'Jadzia' na 'Jadwiga'."
                                      f"Zwróć wynik w formacie json z polem Imię i Nazwisko."},
        {"role": "user", "content": f"Pytanie: ###{question}###"}
    ],
    )

In [4]:
filter_personal_data = response.choices[0].message.content
json_personal_data = json.loads(filter_personal_data)
filtered_people = [person for person in people_data if person.get('imie') == json_personal_data['Imię'] and
                   person.get('nazwisko') == json_personal_data['Nazwisko']]

print(f'Filtered people_data({len(people_data)}) to {len(filtered_people)} pearson(s) based on: {json_personal_data}')

Filtered people_data(1387) to 1 pearson(s) based on: {'Imię': 'Mariusz', 'Nazwisko': 'Kaczor'}


In [5]:
# ask LLM about finding answer
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": f"Odpowiedź na poniższe pytanie na podstawie tej wiedzy: ### {filtered_people} ###"},
        {"role": "user", "content": f"Pytanie: ###{question}###"}
    ],
    )

answer = response2.choices[0].message.content
print(f'question: {question}')
print(f'answer: {answer}')

send_answer(token, answer)

question: jaki kolor się podoba Mariuszowi Kaczorowi?
answer: Mariuszowi Kaczorowi podoba się kolor koralowy.
Sending answer: done! :)
